# Neuro Painter™

A deep, dense, and dumb neural network that learns to paint by pixel.

## Imports

In [ ]:
import os
import time
import json
import random
import numpy as np
import tensorflow as tf

from libs import utils, gif
from libs.group_norm import GroupNormalization

from keras.models import Model, load_model, model_from_json
from keras.layers import Input, Flatten, Reshape, Add, Multiply, Activation, Lambda
from keras.layers import Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization
from keras.objectives import mean_squared_error, mean_absolute_error
from keras.callbacks import ModelCheckpoint, LambdaCallback

from keras import optimizers
from keras import backend as K

## Load Image

In [ ]:
DIRECTORY = 'roadtrip'
IMAGE = "52.jpg"
SIZE = 256
OUTPUT_SIZE = 256
CHANNELS = 3

FILENAME = DIRECTORY + '/' + IMAGE
FEATURES = SIZE*SIZE*CHANNELS
MODEL_NAME = DIRECTORY + '-' + os.path.splitext(IMAGE)[0] +'-'+str(SIZE)

print(MODEL_NAME)

In [ ]:
XS, YS, IMG = utils.load_image(FILENAME, SIZE, SIZE)

print("xs: ", XS.shape)
print("ys: ", YS.shape)
print("img: ", IMG.shape)

## Callbacks 

In [ ]:
GIF_STEPS = 10

def gifit(epoch=None):
    if(epoch < 100):
        GIF_STEPS = 1
    elif(epoch < 1000):
        GIF_STEPS = 10
    else:
        GIF_STEPS = 100
        
    if (epoch % GIF_STEPS == 0):
        print('saving gif ...')
        PREDICTION = NEUROPAINTER.predict_on_batch(XS)
        RECONS.append(np.clip(PREDICTION.reshape((OUTPUT_SIZE, OUTPUT_SIZE, 3)), 0, 1))
        
        
def saveit(epoch=None):
    if (epoch == 0):
        print('saving model ...')
        with open(MODEL_NAME+'-model.json', 'w') as f:
            json.dump(NEUROPAINTER.to_json(), f, ensure_ascii=False)
            
    if (epoch % MODEL_STEPS == 0):
        print('saving weights ...')
        NEUROPAINTER.save_weights(MODEL_NAME+'-weights.h5')

## Training 

In [ ]:
N_NEURONS = 128
N_LAYERS  = 32

INITIALIZER = 'he_normal'
ACTIVATION  = 'elu'

EPOCHS = 20000
BATCH_SIZE = 1024

In [ ]:
RECONS = []

# input
X = Input(shape=(2,))

# set the current layer
current_layer = X

# build hidden layers
for i in range(N_LAYERS):
    current_layer = Dense(N_NEURONS,activation=ACTIVATION,kernel_initializer=INITIALIZER)(current_layer)
    
# output
Y_PRED = Dense(3,activation=None,kernel_initializer=INITIALIZER)(current_layer)

# define model
NEUROPAINTER = Model(inputs=[X], outputs=[Y_PRED])

# define optimizer
ADAM = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=True)

# compile model
NEUROPAINTER.compile(optimizer=ADAM,loss='mean_squared_error')

# print summary
NEUROPAINTER.summary()

# callbacks
giffer = LambdaCallback(on_epoch_end=lambda epoch, logs: gifit(epoch))
saver = LambdaCallback(on_epoch_end=lambda epoch, logs: saveit(epoch))

# fit model
NEUROPAINTER.fit(x=XS,y=YS,batch_size=BATCH_SIZE,epochs=EPOCHS,shuffle=True,callbacks=[giffer,saver],verbose=1)

# training gif
gif.build_gif(RECONS, show_gif=False)